In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda, GRU, BatchNormalization, Bidirectional
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD, RMSprop
import math
import pickle
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

In [5]:
def calculate_class(returns):
    """Find the class for each LSTM sequence based on the median returns."""
    median_returns = returns.median(axis=1)
    labels = returns.iloc[:, :].apply(lambda x: np.where
                                      (x >= median_returns, 1, 0), axis=0)
    return labels

In [6]:
np.random.seed(2)
feature = 31
x_train = np.random.rand(750, feature)
x_train.shape

(750, 31)

In [7]:
y_train = to_categorical(calculate_class(pd.DataFrame(x_train)).values, 2)
y_train.shape

(750, 31, 2)

In [8]:
x_test = np.random.rand(750, feature)
y_test = to_categorical(calculate_class(pd.DataFrame(x_test)).values, 2)

In [9]:
x_train[0]

array([0.4359949 , 0.02592623, 0.54966248, 0.43532239, 0.4203678 ,
       0.33033482, 0.20464863, 0.61927097, 0.29965467, 0.26682728,
       0.62113383, 0.52914209, 0.13457995, 0.51357812, 0.18443987,
       0.78533515, 0.85397529, 0.49423684, 0.84656149, 0.07964548,
       0.50524609, 0.0652865 , 0.42812233, 0.09653092, 0.12715997,
       0.59674531, 0.226012  , 0.10694568, 0.22030621, 0.34982629,
       0.46778748])

In [10]:
x_test[0]

array([0.14867022, 0.13899823, 0.10494235, 0.87939913, 0.18732867,
       0.22262717, 0.7317139 , 0.77066332, 0.10318812, 0.38814662,
       0.56174004, 0.35915058, 0.41018272, 0.8014471 , 0.81498221,
       0.87985186, 0.85469715, 0.81734218, 0.66587059, 0.85641202,
       0.54491559, 0.67412301, 0.34791387, 0.87840982, 0.60886867,
       0.4042137 , 0.12943719, 0.66850456, 0.93534669, 0.88344742,
       0.57987801])

In [11]:
timestep = 240

In [12]:
x_series = [x_train[i:i+timestep, :] for i in range(1, x_train.shape[0] - timestep)]
y_series = [y_train[i+timestep] for i in range(y_train.shape[0] - timestep - 1)]
x = np.array(x_series)
y = np.array(y_series)
print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")

x shape: (509, 240, 31)
y shape: (509, 31, 2)


In [28]:
x_series1 = [x_test[i:i+timestep, :] for i in range(1, x_test.shape[0] - timestep)]
y_series1 = [y_test[i+timestep] for i in range(y_test.shape[0] - timestep - 1)]
x1 = np.array(x_series1)
y1 = np.array(y_series1)
print(f"x1 shape: {x1.shape}")
print(f"y1 shape: {y1.shape}")

x1 shape: (509, 240, 31)
y1 shape: (509, 31, 2)


In [24]:
regressor = Sequential()
regressor.add(LSTM(units=25, input_shape=(timestep, feature)))
regressor.add(Dense(feature * 2, activation='relu'))
regressor.add(Reshape((feature, 2)))
# regressor.add(Lambda(lambda x: softmax(x, axis=-1)))
regressor.add(Dense(2, activation='softmax'))
regressor.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
regressor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 25)                5700      
_________________________________________________________________
dense_3 (Dense)              (None, 62)                1612      
_________________________________________________________________
reshape_2 (Reshape)          (None, 31, 2)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 31, 2)             6         
Total params: 7,318
Trainable params: 7,318
Non-trainable params: 0
_________________________________________________________________


In [27]:
# result = regressor.fit_generator(train_gen, steps_per_epoch=len(train_gen), epochs=1000)
result = regressor.fit(x, y, epochs=100, validation_data=(x1, y1), callbacks = [EarlyStopping(monitor='val_loss', mode='min', patience=10),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_acc', save_best_only=True)])

Train on 509 samples, validate on 509 samples
Epoch 1/100
509/509 [==============================] - 4s 7ms/step - loss: 0.4044 - acc: 0.8138 - val_loss: 0.4569 - val_acc: 0.7725
Epoch 2/100
509/509 [==============================] - 4s 7ms/step - loss: 0.4009 - acc: 0.8156 - val_loss: 0.4536 - val_acc: 0.7772
Epoch 3/100
509/509 [==============================] - 4s 7ms/step - loss: 0.3970 - acc: 0.8186 - val_loss: 0.4505 - val_acc: 0.7782
Epoch 4/100
384/509 [=====================>........] - ETA: 0s - loss: 0.3954 - acc: 0.8165

KeyboardInterrupt: 

In [26]:
plt.plot(result.history["acc"])
plt.plot(result.history["val_acc"])

NameError: name 'result' is not defined

In [ ]:
plt.plot(result.history["loss"])
plt.plot(result.history["val_loss"])